# Logistic Regression

Objectives:
1. Define a classification problem
2. Understand why linear regression does not fit to solve classification problem
3. Understand logistic function
4. Demonstrate the cost function
5. Apply gradient descent with logistic function

In [3]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from class_utils.lab_utils_common import dlc, plot_data, draw_vthresh, sigmoid
from class_utils.plt_one_addpt_onclick import plt_one_addpt_onclick
from class_utils.plt_logistic_loss import  plt_logistic_cost, plt_two_logistic_loss_curves, plt_simple_example
from class_utils.plt_logistic_loss import soup_bowl, plt_logistic_squared_error

%matplotlib notebook
%matplotlib inline
plt.style.use('../Style/deeplearning.mplstyle')